Django文件上传

```python
# encoding: utf8

import logging
import os.path

from django.conf import settings
from rest_framework import generics
from rest_framework.response import Response
from rest_framework import serializers as base_serializers
from drf_yasg2.utils import swagger_auto_schema

from libs.rest_framework.exceptions import SUCCESS_CODE, ERROR_CODES
from utils.upload import S3Upload

from . import serializers

logger = logging.getLogger('app')


class FileUploadView(generics.GenericAPIView):

    serializer_class = base_serializers.Serializer

    @swagger_auto_schema(responses={'200': serializers.FileUploadResponseSerializer})
    def post(self, request, *args, **kwargs):   # noqa

        urls = []
        # 遍历上传的文件列表
        for name in request.FILES:
            # in_file为`django.core.files.uploadedfile.InMemoryUploadedFile`实例
            in_file = request.FILES[name]

            # 将文件导出为本地文件
            output_file_path = f'/tmp/{in_file.name}'
            with open(output_file_path, 'wb') as output_file:
                for chunk in in_file.chunks():
                    output_file.write(chunk)

            # 文件上传到S3
            upload_s3_url = S3Upload().upload(output_file_path)
            urls.append(upload_s3_url)

            # 删除本地文件
            if os.path.exists(output_file_path):
                os.remove(output_file_path)

        resp_data = {
            'code': SUCCESS_CODE,
            'message': ERROR_CODES[SUCCESS_CODE],
            'result': {
                'urls': urls
            }
        }
        return Response(resp_data)
    
```
